#### Aquí encontrarás todo lo que necesitas saber: https://www.kaggle.com/competitions/desafio-de-prediccion-de-precios-de-portatiles

## Librerias

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import Toolbox_ML as TB
import bootcampviztools as bt
import re
import os

from sklearn import metrics
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from lightgbm import LGBMRegressor
from sklearn.metrics import mean_absolute_error

import warnings
warnings.filterwarnings(action="ignore", message="^internal gelsd")

## Data

In [3]:
### Para que funcione necesitas bajarte los archivos de datos de Kaggle 
df_train = pd.read_csv ('.\desafio-de-prediccion-de-precios-de-portatiles/train.csv')
df_test = pd.read_csv ('.\desafio-de-prediccion-de-precios-de-portatiles/test.csv')

In [4]:
print (df_train.shape)
print (df_test.shape)

(912, 14)
(391, 13)


## Data exploration

In [5]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 912 entries, 0 to 911
Data columns (total 14 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   id                912 non-null    int64  
 1   laptop_ID         912 non-null    int64  
 2   Company           912 non-null    object 
 3   Product           912 non-null    object 
 4   TypeName          912 non-null    object 
 5   Inches            912 non-null    float64
 6   ScreenResolution  912 non-null    object 
 7   Cpu               912 non-null    object 
 8   Ram               912 non-null    object 
 9   Memory            912 non-null    object 
 10  Gpu               912 non-null    object 
 11  OpSys             912 non-null    object 
 12  Weight            912 non-null    object 
 13  Price_euros       912 non-null    float64
dtypes: float64(2), int64(2), object(10)
memory usage: 99.9+ KB


In [6]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 391 entries, 0 to 390
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   id                391 non-null    int64  
 1   laptop_ID         391 non-null    int64  
 2   Company           391 non-null    object 
 3   Product           391 non-null    object 
 4   TypeName          391 non-null    object 
 5   Inches            391 non-null    float64
 6   ScreenResolution  391 non-null    object 
 7   Cpu               391 non-null    object 
 8   Ram               391 non-null    object 
 9   Memory            391 non-null    object 
 10  Gpu               391 non-null    object 
 11  OpSys             391 non-null    object 
 12  Weight            391 non-null    object 
dtypes: float64(1), int64(2), object(10)
memory usage: 39.8+ KB


En test solo falta la columna target "Price_euros"

In [7]:
df_train.describe()

,id,laptop_ID,Inches,Price_euros
count,912.000000,912.000000,912.000000,912.000000
mean,645.091009,666.192982,15.011404,1108.122873
std,380.313813,384.873846,1.411744,714.597741
min,0.000000,1.000000,10.100000,174.000000
25%,319.750000,330.500000,14.000000,589.000000
50%,636.500000,673.500000,15.600000,949.000000
75%,980.250000,998.500000,15.600000,1458.500000
max,1302.000000,1320.000000,18.400000,6099.000000


In [8]:
TB.describe_df(df_train)

,id,laptop_ID,Company,Product,TypeName,Inches,ScreenResolution,Cpu,Ram,Memory,Gpu,OpSys,Weight,Price_euros
Tipos,int64,int64,object,object,object,float64,object,object,object,object,object,object,object,float64
% Faltante,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Valores Únicos,912,912,19,475,6,15,35,104,8,36,91,9,158,614
% Cardinalidad,100.0,100.0,2.08,52.08,0.66,1.64,3.84,11.4,0.88,3.95,9.98,0.99,17.32,67.32


In [9]:
TB.tipifica_variables (df_train, 10, 30)

,nombre_variable,tipo_sugerido
0,id,Numérica Continua
1,laptop_ID,Numérica Continua
2,Company,Numérica Discreta
3,Product,Numérica Continua
4,TypeName,Categórica
5,Inches,Numérica Discreta
6,ScreenResolution,Numérica Discreta
7,Cpu,Numérica Discreta
8,Ram,Categórica
9,Memory,Numérica Discreta


In [10]:
df_train.head (10)

,id,laptop_ID,Company,Product,TypeName,Inches,ScreenResolution,Cpu,Ram,Memory,Gpu,OpSys,Weight,Price_euros
0,268,1002,Dell,Inspiron 5567,Notebook,15.6,1366x768,Intel Core i7 7500U 2.7GHz,8GB,1TB HDD,AMD Radeon R7 M445,Windows 10,2.36kg,749.00
1,347,867,Asus,X541NA (N4200/4GB/1TB/W10),Notebook,15.6,1366x768,Intel Pentium Quad Core N4200 1.1GHz,4GB,1TB HDD,Intel HD Graphics 505,Windows 10,2kg,449.00
2,353,966,Toshiba,Portege Z30-C-1CW,Notebook,13.3,Full HD 1920x1080,Intel Core i5 6200U 2.3GHz,8GB,256GB SSD,Intel HD Graphics 520,Windows 7,1.2kg,1460.00
3,578,767,Dell,Alienware 17,Gaming,15.6,IPS Panel 4K Ultra HD 3840x2160,Intel Core i7 7700HQ 2.8GHz,16GB,256GB SSD + 1TB HDD,Nvidia GeForce GTX 1070,Windows 10,4.42kg,2868.99
4,42,1241,Dell,Latitude E7270,Ultrabook,12.5,Full HD / Touchscreen 1920x1080,Intel Core i5 6300U 2.4GHz,8GB,256GB SSD,Intel HD Graphics 520,Windows 7,1.26kg,1713.37
5,867,376,Asus,ZenBook Flip,2 in 1 Convertible,15.6,Full HD / Touchscreen 1920x1080,Intel Core i7 7500U 2.7GHz,12GB,512GB SSD + 2TB HDD,Nvidia GeForce GT 940MX,Windows 10,2.26kg,1099.00
6,1136,60,Asus,X541UA-DM1897 (i3-6006U/4GB/256GB/FHD/Linux),Notebook,15.6,Full HD 1920x1080,Intel Core i3 6006U 2GHz,4GB,256GB SSD,Intel HD Graphics 520,Linux,2kg,415.00
7,609,1157,MSI,GP62M 7RDX,Gaming,15.6,Full HD 1920x1080,Intel Core i7 7700HQ 2.8GHz,8GB,128GB SSD + 1TB HDD,Nvidia GeForce GTX 1050,Windows 10,2.4kg,1142.80
8,474,110,Asus,UX430UQ-GV209R (i7-7500U/8GB/256GB/GeForce,Ultrabook,14.0,IPS Panel Full HD 1920x1080,Intel Core i7 7500U 2.7GHz,8GB,256GB SSD,Nvidia GeForce 940MX,Windows 10,1.3kg,1193.00
9,838,1263,Asus,Rog GL553VE-FY052T,Gaming,15.6,IPS Panel Full HD 1920x1080,Intel Core i7 7700HQ 2.8GHz,16GB,256GB SSD + 1TB HDD,Nvidia GeForce GTX 1050 Ti,Windows 10,2.5kg,1600.00


## Data processing

In [11]:
print (df_test.columns)

Index(['id', 'laptop_ID', 'Company', 'Product', 'TypeName', 'Inches',
       'ScreenResolution', 'Cpu', 'Ram', 'Memory', 'Gpu', 'OpSys', 'Weight'],
      dtype='object')


In [12]:
df_train.ScreenResolution.value_counts()

ScreenResolution
Full HD 1920x1080                                346
1366x768                                         208
IPS Panel Full HD 1920x1080                      163
IPS Panel Full HD / Touchscreen 1920x1080         37
Full HD / Touchscreen 1920x1080                   32
Touchscreen 1366x768                              15
1600x900                                          11
IPS Panel 4K Ultra HD 3840x2160                   10
4K Ultra HD / Touchscreen 3840x2160                8
Quad HD+ / Touchscreen 3200x1800                   8
IPS Panel 4K Ultra HD / Touchscreen 3840x2160      8
IPS Panel Retina Display 2304x1440                 6
IPS Panel Retina Display 2560x1600                 5
IPS Panel 1366x768                                 5
4K Ultra HD 3840x2160                              5
IPS Panel Touchscreen 1920x1200                    4
Touchscreen 2560x1440                              4
IPS Panel Quad HD+ / Touchscreen 3200x1800         4
IPS Panel Touchscreen 2560x14

In [13]:
"""# convertir todos los nombres a minusculas
df_train.columns = df_train.columns.str.lower()
df_test.columns = df_test.columns.str.lower()"""

'# convertir todos los nombres a minusculas\ndf_train.columns = df_train.columns.str.lower()\ndf_test.columns = df_test.columns.str.lower()'

In [14]:
# Columna "ScreenResolution"
# hay que extraer el ancho y el alto de la pantalla.

def extract_resolution(screen_resolution):
    print(f"Procesando: {screen_resolution}")  # Mensaje de depuración
    if pd.isna(screen_resolution):
        return None
    match = re.search(r'(\d{3,4}x\d{3,4})', str(screen_resolution))
    if match:
        print(f"Encontrado: {match.group(1)}")  # Mensaje de depuración
        return match.group(1)
    return None

# Aplicar la función a la columna 'ScreenResolution' en el conjunto de entrenamiento
df_train['ScreenResolution'] = df_train['ScreenResolution'].apply(extract_resolution)

# Eliminar las filas donde no se encontró ninguna resolución numérica en el conjunto de entrenamiento
train = df_train.dropna(subset=['ScreenResolution'])

# Aplicar la función a la columna 'ScreenResolution' en el conjunto de prueba
df_test['ScreenResolution'] = df_test['ScreenResolution'].apply(extract_resolution)

# Eliminar las filas donde no se encontró ninguna resolución numérica en el conjunto de prueba
test = df_test.dropna(subset=['ScreenResolution'])

# Mostrar los primeros valores para verificar
print(train['ScreenResolution'].head())
print(test['ScreenResolution'].head())

Procesando: 1366x768
Encontrado: 1366x768
Procesando: 1366x768
Encontrado: 1366x768
Procesando: Full HD 1920x1080
Encontrado: 1920x1080
Procesando: IPS Panel 4K Ultra HD 3840x2160
Encontrado: 3840x2160
Procesando: Full HD / Touchscreen 1920x1080
Encontrado: 1920x1080
Procesando: Full HD / Touchscreen 1920x1080
Encontrado: 1920x1080
Procesando: Full HD 1920x1080
Encontrado: 1920x1080
Procesando: Full HD 1920x1080
Encontrado: 1920x1080
Procesando: IPS Panel Full HD 1920x1080
Encontrado: 1920x1080
Procesando: IPS Panel Full HD 1920x1080
Encontrado: 1920x1080
Procesando: 1366x768
Encontrado: 1366x768
Procesando: Full HD 1920x1080
Encontrado: 1920x1080
Procesando: 1366x768
Encontrado: 1366x768
Procesando: IPS Panel Full HD 1920x1080
Encontrado: 1920x1080
Procesando: 1366x768
Encontrado: 1366x768
Procesando: IPS Panel Full HD 1920x1080
Encontrado: 1920x1080
Procesando: IPS Panel Full HD / Touchscreen 1920x1080
Encontrado: 1920x1080
Procesando: 1366x768
Encontrado: 1366x768
Procesando: IPS Pa

In [15]:
train.head()

,id,laptop_ID,Company,Product,TypeName,Inches,ScreenResolution,Cpu,Ram,Memory,Gpu,OpSys,Weight,Price_euros
0,268,1002,Dell,Inspiron 5567,Notebook,15.6,1366x768,Intel Core i7 7500U 2.7GHz,8GB,1TB HDD,AMD Radeon R7 M445,Windows 10,2.36kg,749.00
1,347,867,Asus,X541NA (N4200/4GB/1TB/W10),Notebook,15.6,1366x768,Intel Pentium Quad Core N4200 1.1GHz,4GB,1TB HDD,Intel HD Graphics 505,Windows 10,2kg,449.00
2,353,966,Toshiba,Portege Z30-C-1CW,Notebook,13.3,1920x1080,Intel Core i5 6200U 2.3GHz,8GB,256GB SSD,Intel HD Graphics 520,Windows 7,1.2kg,1460.00
3,578,767,Dell,Alienware 17,Gaming,15.6,3840x2160,Intel Core i7 7700HQ 2.8GHz,16GB,256GB SSD + 1TB HDD,Nvidia GeForce GTX 1070,Windows 10,4.42kg,2868.99
4,42,1241,Dell,Latitude E7270,Ultrabook,12.5,1920x1080,Intel Core i5 6300U 2.4GHz,8GB,256GB SSD,Intel HD Graphics 520,Windows 7,1.26kg,1713.37


In [16]:
# Columna "Cpu"
# hay que extraer el fabricante y el modelo del procesador.

def extract_from_Cpu(train):
    train["Vel"] = train["Cpu"].str.extract(r'(\d+(?:\.\d+)?GHz)')
    train["Vel"] = train["Vel"].str.replace('GHz','')
    train["Vel"] = train["Vel"].astype(float)
    train["Cpu"] = train["Cpu"].str.replace(r'(\d+(?:\.\d+)?GHz)','',regex =True)
    

extract_from_Cpu (test)
extract_from_Cpu (train)

In [17]:
train.head()

,id,laptop_ID,Company,Product,TypeName,Inches,ScreenResolution,Cpu,Ram,Memory,Gpu,OpSys,Weight,Price_euros,Vel
0,268,1002,Dell,Inspiron 5567,Notebook,15.6,1366x768,Intel Core i7 7500U,8GB,1TB HDD,AMD Radeon R7 M445,Windows 10,2.36kg,749.00,2.7
1,347,867,Asus,X541NA (N4200/4GB/1TB/W10),Notebook,15.6,1366x768,Intel Pentium Quad Core N4200,4GB,1TB HDD,Intel HD Graphics 505,Windows 10,2kg,449.00,1.1
2,353,966,Toshiba,Portege Z30-C-1CW,Notebook,13.3,1920x1080,Intel Core i5 6200U,8GB,256GB SSD,Intel HD Graphics 520,Windows 7,1.2kg,1460.00,2.3
3,578,767,Dell,Alienware 17,Gaming,15.6,3840x2160,Intel Core i7 7700HQ,16GB,256GB SSD + 1TB HDD,Nvidia GeForce GTX 1070,Windows 10,4.42kg,2868.99,2.8
4,42,1241,Dell,Latitude E7270,Ultrabook,12.5,1920x1080,Intel Core i5 6300U,8GB,256GB SSD,Intel HD Graphics 520,Windows 7,1.26kg,1713.37,2.4


In [18]:
# Procesar la columna "Ram"
# tiene el sufijo GB, hay que eliminarlo y convertirlo a númerico

train['Ram'] = train['Ram'].str.replace('GB', '').astype(int)

# para df_test
test['Ram'] = test['Ram'].str.replace('GB', '').astype(int)

In [19]:
train.head()

,id,laptop_ID,Company,Product,TypeName,Inches,ScreenResolution,Cpu,Ram,Memory,Gpu,OpSys,Weight,Price_euros,Vel
0,268,1002,Dell,Inspiron 5567,Notebook,15.6,1366x768,Intel Core i7 7500U,8,1TB HDD,AMD Radeon R7 M445,Windows 10,2.36kg,749.00,2.7
1,347,867,Asus,X541NA (N4200/4GB/1TB/W10),Notebook,15.6,1366x768,Intel Pentium Quad Core N4200,4,1TB HDD,Intel HD Graphics 505,Windows 10,2kg,449.00,1.1
2,353,966,Toshiba,Portege Z30-C-1CW,Notebook,13.3,1920x1080,Intel Core i5 6200U,8,256GB SSD,Intel HD Graphics 520,Windows 7,1.2kg,1460.00,2.3
3,578,767,Dell,Alienware 17,Gaming,15.6,3840x2160,Intel Core i7 7700HQ,16,256GB SSD + 1TB HDD,Nvidia GeForce GTX 1070,Windows 10,4.42kg,2868.99,2.8
4,42,1241,Dell,Latitude E7270,Ultrabook,12.5,1920x1080,Intel Core i5 6300U,8,256GB SSD,Intel HD Graphics 520,Windows 7,1.26kg,1713.37,2.4


In [20]:
train.Memory.value_counts()

Memory
256GB SSD                        293
1TB HDD                          150
500GB HDD                        100
512GB SSD                         75
128GB SSD +  1TB HDD              61
128GB SSD                         56
256GB SSD +  1TB HDD              52
32GB Flash Storage                27
64GB Flash Storage                11
2TB HDD                           11
512GB SSD +  1TB HDD              10
1TB SSD                            9
1.0TB Hybrid                       7
256GB Flash Storage                7
32GB SSD                           5
180GB SSD                          5
256GB SSD +  2TB HDD               4
16GB Flash Storage                 4
16GB SSD                           3
256GB SSD +  500GB HDD             2
1TB SSD +  1TB HDD                 2
128GB Flash Storage                2
512GB SSD +  2TB HDD               2
512GB Flash Storage                2
1.0TB HDD                          1
32GB HDD                           1
128GB SSD +  2TB HDD           

In [21]:
# Columna "Memory"
# Hay que separar los tipos y tamaños de almacenamiento.

# Definir una función para convertir tamaños a GB
def convert_to_gb(size, unit):
    if unit == 'tb':
        return float(size) * 1024
    elif unit == 'gb':
        return float(size)
    return 0

# Definir una función para procesar la columna 'Memory'
def process_memory(memory):
    memory = memory.lower().strip()
    parts = re.split(r'\s*\+\s*', memory)
    
    ssd = 0
    hdd = 0
    flash = 0
    hybrid = 0
    
    for part in parts:
        match = re.match(r'(\d+(?:\.\d+)?)\s*(tb|gb)\s*(ssd|hdd|flash|hybrid)?', part)
        if match:
            size, unit, storage_type = match.groups()
            size_in_gb = convert_to_gb(size, unit)
            if storage_type == 'ssd':
                ssd += size_in_gb
            elif storage_type == 'hdd':
                hdd += size_in_gb
            elif storage_type == 'flash':
                flash += size_in_gb
            elif storage_type == 'hybrid':
                hybrid += size_in_gb
        else:
            print(f"No se pudo procesar: {part}")
    
    return ssd, hdd, flash, hybrid


In [22]:
# Aplicar la función a la columna 'Memory' en el conjunto de entrenamiento
train[['SSD', 'HDD', 'Flash', 'Hybrid']] = train['Memory'].apply(process_memory).apply(pd.Series)

# Aplicar la función a la columna 'Memory' en el conjunto de prueba
test[['SSD', 'HDD', 'Flash', 'Hybrid']] = test['Memory'].apply(process_memory).apply(pd.Series)

# Mostrar los primeros valores para verificar
print(train[['Memory', 'SSD', 'HDD', 'Flash', 'Hybrid']].head())
print(test[['Memory', 'SSD', 'HDD', 'Flash', 'Hybrid']].head())

                 Memory    SSD     HDD  Flash  Hybrid
0               1TB HDD    0.0  1024.0    0.0     0.0
1               1TB HDD    0.0  1024.0    0.0     0.0
2             256GB SSD  256.0     0.0    0.0     0.0
3  256GB SSD +  1TB HDD  256.0  1024.0    0.0     0.0
4             256GB SSD  256.0     0.0    0.0     0.0
      Memory    SSD    HDD  Flash  Hybrid
0  512GB SSD  512.0    0.0    0.0     0.0
1  256GB SSD  256.0    0.0    0.0     0.0
2  500GB HDD    0.0  500.0    0.0     0.0
3  256GB SSD  256.0    0.0    0.0     0.0
4  256GB SSD  256.0    0.0    0.0     0.0


In [23]:
train.head()

,id,laptop_ID,Company,Product,TypeName,Inches,ScreenResolution,Cpu,Ram,Memory,Gpu,OpSys,Weight,Price_euros,Vel,SSD,HDD,Flash,Hybrid
0,268,1002,Dell,Inspiron 5567,Notebook,15.6,1366x768,Intel Core i7 7500U,8,1TB HDD,AMD Radeon R7 M445,Windows 10,2.36kg,749.00,2.7,0.0,1024.0,0.0,0.0
1,347,867,Asus,X541NA (N4200/4GB/1TB/W10),Notebook,15.6,1366x768,Intel Pentium Quad Core N4200,4,1TB HDD,Intel HD Graphics 505,Windows 10,2kg,449.00,1.1,0.0,1024.0,0.0,0.0
2,353,966,Toshiba,Portege Z30-C-1CW,Notebook,13.3,1920x1080,Intel Core i5 6200U,8,256GB SSD,Intel HD Graphics 520,Windows 7,1.2kg,1460.00,2.3,256.0,0.0,0.0,0.0
3,578,767,Dell,Alienware 17,Gaming,15.6,3840x2160,Intel Core i7 7700HQ,16,256GB SSD + 1TB HDD,Nvidia GeForce GTX 1070,Windows 10,4.42kg,2868.99,2.8,256.0,1024.0,0.0,0.0
4,42,1241,Dell,Latitude E7270,Ultrabook,12.5,1920x1080,Intel Core i5 6300U,8,256GB SSD,Intel HD Graphics 520,Windows 7,1.26kg,1713.37,2.4,256.0,0.0,0.0,0.0


In [24]:
train.Weight.value_counts()

Weight
2.2kg     86
2.1kg     40
2.3kg     35
2.4kg     31
2kg       30
          ..
4.0kg      1
0.98kg     1
3.6kg      1
2.99kg     1
1.79kg     1
Name: count, Length: 158, dtype: int64

In [25]:
# Columna 'Weight'
# tiene el sufijo kg, hay que eliminarlo y convertirlo a númerico
train['Weight'] = train['Weight'].str.replace('kg', '').astype(float)

# para df_test
test['Weight'] = test['Weight'].str.replace('kg', '').astype(float)

In [26]:
train.head()

,id,laptop_ID,Company,Product,TypeName,Inches,ScreenResolution,Cpu,Ram,Memory,Gpu,OpSys,Weight,Price_euros,Vel,SSD,HDD,Flash,Hybrid
0,268,1002,Dell,Inspiron 5567,Notebook,15.6,1366x768,Intel Core i7 7500U,8,1TB HDD,AMD Radeon R7 M445,Windows 10,2.36,749.00,2.7,0.0,1024.0,0.0,0.0
1,347,867,Asus,X541NA (N4200/4GB/1TB/W10),Notebook,15.6,1366x768,Intel Pentium Quad Core N4200,4,1TB HDD,Intel HD Graphics 505,Windows 10,2.00,449.00,1.1,0.0,1024.0,0.0,0.0
2,353,966,Toshiba,Portege Z30-C-1CW,Notebook,13.3,1920x1080,Intel Core i5 6200U,8,256GB SSD,Intel HD Graphics 520,Windows 7,1.20,1460.00,2.3,256.0,0.0,0.0,0.0
3,578,767,Dell,Alienware 17,Gaming,15.6,3840x2160,Intel Core i7 7700HQ,16,256GB SSD + 1TB HDD,Nvidia GeForce GTX 1070,Windows 10,4.42,2868.99,2.8,256.0,1024.0,0.0,0.0
4,42,1241,Dell,Latitude E7270,Ultrabook,12.5,1920x1080,Intel Core i5 6300U,8,256GB SSD,Intel HD Graphics 520,Windows 7,1.26,1713.37,2.4,256.0,0.0,0.0,0.0


-----------------------------------------------------------------------------------------------------------------

In [27]:
train.Gpu.value_counts()

Gpu
Intel HD Graphics 620      199
Intel HD Graphics 520      138
Intel UHD Graphics 620      45
Nvidia GeForce GTX 1050     43
Nvidia GeForce GTX 1060     31
                          ... 
Nvidia GeForce 920MX         1
Intel HD Graphics 620        1
AMD Radeon R7 M365X          1
Nvidia Quadro M2200M         1
Nvidia Quadro M520M          1
Name: count, Length: 91, dtype: int64

In [28]:
# Definir una función para procesar la columna 'Gpu'
def process_gpu(gpu):
    gpu = gpu.lower().strip()
    # Extraer la marca del GPU
    if 'nvidia' in gpu:
        brand = 'nvidia'
    elif 'amd' in gpu:
        brand = 'amd'
    elif 'intel' in gpu:
        brand = 'intel'
    else:
        brand = 'other'
    
    # Extraer el modelo del GPU
    model = re.sub(r'\b(?:intel|nvidia|amd|graphics|geforce|radeon|uhd|hd|gtx|quadro|rx)\b', '', gpu).strip()
    
    return brand, model

In [29]:
# Aplicar la función a la columna 'Gpu' en el conjunto de entrenamiento
train[['Gpu_Brand', 'Gpu_Model']] = train['Gpu'].apply(process_gpu).apply(pd.Series)

# Aplicar la función a la columna 'Gpu' en el conjunto de prueba
test[['Gpu_Brand', 'Gpu_Model']] = test['Gpu'].apply(process_gpu).apply(pd.Series)

# Mostrar los primeros valores para verificar
print(train[['Gpu', 'Gpu_Brand', 'Gpu_Model']].head())
print(test[['Gpu', 'Gpu_Brand', 'Gpu_Model']].head())

                       Gpu Gpu_Brand Gpu_Model
0       AMD Radeon R7 M445       amd   r7 m445
1    Intel HD Graphics 505     intel       505
2    Intel HD Graphics 520     intel       520
3  Nvidia GeForce GTX 1070    nvidia      1070
4    Intel HD Graphics 520     intel       520
                     Gpu Gpu_Brand Gpu_Model
0  Intel HD Graphics 620     intel       620
1      AMD Radeon RX 540       amd       540
2  Intel HD Graphics 520     intel       520
3  Intel HD Graphics 520     intel       520
4  Intel HD Graphics 620     intel       620


In [30]:
train.Gpu_Brand.value_counts()

Gpu_Brand
intel     522
nvidia    268
amd       122
Name: count, dtype: int64

In [31]:
#train.Gpu_Model.value_counts()

In [32]:
test.columns

Index(['id', 'laptop_ID', 'Company', 'Product', 'TypeName', 'Inches',
       'ScreenResolution', 'Cpu', 'Ram', 'Memory', 'Gpu', 'OpSys', 'Weight',
       'Vel', 'SSD', 'HDD', 'Flash', 'Hybrid', 'Gpu_Brand', 'Gpu_Model'],
      dtype='object')

In [33]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 912 entries, 0 to 911
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   id                912 non-null    int64  
 1   laptop_ID         912 non-null    int64  
 2   Company           912 non-null    object 
 3   Product           912 non-null    object 
 4   TypeName          912 non-null    object 
 5   Inches            912 non-null    float64
 6   ScreenResolution  912 non-null    object 
 7   Cpu               912 non-null    object 
 8   Ram               912 non-null    int32  
 9   Memory            912 non-null    object 
 10  Gpu               912 non-null    object 
 11  OpSys             912 non-null    object 
 12  Weight            912 non-null    float64
 13  Price_euros       912 non-null    float64
 14  Vel               912 non-null    float64
 15  SSD               912 non-null    float64
 16  HDD               912 non-null    float64
 1

## Pasos que si o si debéis realizar para poder participar en la competición:

In [34]:
train.columns

Index(['id', 'laptop_ID', 'Company', 'Product', 'TypeName', 'Inches',
       'ScreenResolution', 'Cpu', 'Ram', 'Memory', 'Gpu', 'OpSys', 'Weight',
       'Price_euros', 'Vel', 'SSD', 'HDD', 'Flash', 'Hybrid', 'Gpu_Brand',
       'Gpu_Model'],
      dtype='object')

### 1. Definir X e y

In [35]:
# Definir X e y
X = train.drop(columns=['laptop_ID', 'Product', 'Memory', 'Weight', 'Price_euros'])
y = train['Price_euros']

# Convertir columnas categóricas a variables dummy
X = pd.get_dummies(X, drop_first=True)


### 2. Dividir X_train, X_test, y_train, y_test

In [36]:
# Dividir en conjuntos de entrenamiento y prueba
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Mostrar la forma de los conjuntos de datos
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((729, 304), (183, 304), (729,), (183,))

### 3. Crear y entrenar el modelo


In [37]:
# Definir el pipeline
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('model', LinearRegression())
])

# Definir la búsqueda de hiperparámetros para los modelos
param_grid = [
    {
        'model': [LinearRegression()],
    },
    {
        'model': [RandomForestRegressor()],
        'model__n_estimators': [100, 200],
        'model__max_depth': [None, 10, 20],
    },
    {
        'model': [LGBMRegressor()],
        'model__num_leaves': [31, 50],
        'model__n_estimators': [100, 200],
    }
]

# Realizar la búsqueda de hiperparámetros
grid_search = GridSearchCV(pipeline, param_grid, cv=3, scoring='neg_mean_absolute_error', verbose=2, n_jobs=-1)
grid_search.fit(X_train, y_train)

Fitting 3 folds for each of 11 candidates, totalling 33 fits


GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('scaler', StandardScaler()),
                                       ('model', LinearRegression())]),
             n_jobs=-1,
             param_grid=[{'model': [LinearRegression()]},
                         {'model': [RandomForestRegressor()],
                          'model__max_depth': [None, 10, 20],
                          'model__n_estimators': [100, 200]},
                         {'model': [LGBMRegressor()],
                          'model__n_estimators': [100, 200],
                          'model__num_leaves': [31, 50]}],
             scoring='neg_mean_absolute_error', verbose=2)

In [38]:
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from lightgbm import LGBMRegressor
from sklearn.svm import SVR
from xgboost import XGBRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_absolute_error

# Definir el pipeline para cada modelo
pipelines = {
    'LinearRegression': Pipeline([
        ('scaler', StandardScaler()),
        ('model', LinearRegression())
    ]),
    'RandomForestRegressor': Pipeline([
        ('scaler', StandardScaler()),
        ('model', RandomForestRegressor())
    ]),
    'LGBMRegressor': Pipeline([
        ('scaler', StandardScaler()),
        ('model', LGBMRegressor())
    ]),
    'SVR': Pipeline([
        ('scaler', StandardScaler()),
        ('model', SVR())
    ]),
    'XGBRegressor': Pipeline([
        ('scaler', StandardScaler()),
        ('model', XGBRegressor())
    ])
}

# Definir la búsqueda de hiperparámetros para cada modelo
param_grids = {
    'LinearRegression': {
        'model': [LinearRegression()]
    },
    'RandomForestRegressor': {
        'model': [RandomForestRegressor()],
        'model__n_estimators': [100, 200],
        'model__max_depth': [None, 10, 20],
    },
    'LGBMRegressor': {
        'model': [LGBMRegressor()],
        'model__num_leaves': [31, 50],
        'model__n_estimators': [100, 200],
    },
    'SVR': {
        'model': [SVR()],
        'model__kernel': ['linear', 'rbf'],
        'model__C': [0.1, 1, 10],
        'model__gamma': ['scale', 'auto']
    },
    'XGBRegressor': {
        'model': [XGBRegressor()],
        'model__n_estimators': [100, 200],
        'model__max_depth': [3, 6, 9],
        'model__learning_rate': [0.01, 0.1, 0.3]
    }
}

# Realizar la búsqueda de hiperparámetros para cada modelo
best_models = {}
for name, pipeline in pipelines.items():
    print(f"Entrenando modelo: {name}")
    grid_search = GridSearchCV(pipeline, param_grids[name], cv=3, scoring='neg_mean_absolute_error', verbose=2, n_jobs=-1)
    grid_search.fit(X_train, y_train)
    best_models[name] = grid_search.best_estimator_
    print(f"Mejores parámetros para {name}: {grid_search.best_params_}")



Entrenando modelo: LinearRegression
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Mejores parámetros para LinearRegression: {'model': LinearRegression()}
Entrenando modelo: RandomForestRegressor
Fitting 3 folds for each of 6 candidates, totalling 18 fits
Mejores parámetros para RandomForestRegressor: {'model': RandomForestRegressor(), 'model__max_depth': 20, 'model__n_estimators': 200}
Entrenando modelo: LGBMRegressor
Fitting 3 folds for each of 4 candidates, totalling 12 fits
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000503 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 445
[LightGBM] [Info] Number of data points in the train set: 729, number of used features: 53
[LightGBM] [Info] Start training from score 1122.043223
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No 

In [39]:
# Evaluar el mejor modelo en el conjunto de prueba
for name, model in best_models.items():
    y_pred = model.predict(X_test)
    mae = mean_absolute_error(y_test, y_pred)
    print(f"MAE para el mejor modelo {name}: {mae}")

MAE para el mejor modelo LinearRegression: 1990307179310709.5
MAE para el mejor modelo RandomForestRegressor: 201.4668323174787
MAE para el mejor modelo LGBMRegressor: 212.8120343611577
MAE para el mejor modelo SVR: 223.5177061078649
MAE para el mejor modelo XGBRegressor: 199.65386376886423


In [40]:
# Mejor modelo
best_model = grid_search.best_estimator_
best_params = grid_search.best_params_


In [41]:
'''from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error

# Entrenar el modelo
model = LinearRegression()
model.fit(X_train, y_train)
'''

'from sklearn.linear_model import LinearRegression\nfrom sklearn.metrics import mean_absolute_error\n\n# Entrenar el modelo\nmodel = LinearRegression()\nmodel.fit(X_train, y_train)\n'

In [42]:
'''from lightgbm import LGBMRegressor
model_lgbm = LGBMRegressor(random_state=42)
model_lgbm.fit(X_train, y_train)'''

'from lightgbm import LGBMRegressor\nmodel_lgbm = LGBMRegressor(random_state=42)\nmodel_lgbm.fit(X_train, y_train)'

In [43]:
'''from sklearn.ensemble import RandomForestRegressor
model_rf = RandomForestRegressor(random_state=42)
model_rf.fit(X_train, y_train)'''

'from sklearn.ensemble import RandomForestRegressor\nmodel_rf = RandomForestRegressor(random_state=42)\nmodel_rf.fit(X_train, y_train)'

### 4. Sacar métricas, valorar el modelo 

Recuerda que en la competición se va a evaluar con la métrica de MAE.

In [44]:
# Evaluar el mejor modelo en el conjunto de prueba
y_pred = best_model.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
print(f'MAE for the best model: {mae}')
print(f'Best parameters: {best_params}')

MAE for the best model: 199.65386376886423
Best parameters: {'model': XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...), 'model__learning_rate': 0.3, 'model__max_depth': 3, 'model__n_estimators': 200}


In [45]:
'''# Evaluar el modelo
y_pred = model.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
print(f'MAE Lineal Regresion: {mae}')'''

"# Evaluar el modelo\ny_pred = model.predict(X_test)\nmae = mean_absolute_error(y_test, y_pred)\nprint(f'MAE Lineal Regresion: {mae}')"

In [46]:
'''y_pred_lgbm = model_lgbm.predict(X_test)
mae_lgbm = mean_absolute_error(y_test, y_pred_lgbm)
print(f'MAE LightGBM: {mae_lgbm}')'''

"y_pred_lgbm = model_lgbm.predict(X_test)\nmae_lgbm = mean_absolute_error(y_test, y_pred_lgbm)\nprint(f'MAE LightGBM: {mae_lgbm}')"

In [47]:
'''y_pred_rf = model_rf.predict(X_test)
mae_rf = mean_absolute_error(y_test, y_pred_rf)
print(f'MAE Random Forest: {mae_rf}')'''

"y_pred_rf = model_rf.predict(X_test)\nmae_rf = mean_absolute_error(y_test, y_pred_rf)\nprint(f'MAE Random Forest: {mae_rf}')"

-----------------------------------------------------------------

## Una vez listo el modelo, toca predecir con el dataset de predicción 

Definición de **modelo que está listo**. 

Tras hacer suficientes pruebas, analizar los datos, hacer feature engineering, probar diferentes modelos con diferentes parámetros, es con este con el que observo mejores métricas y menos overfitting. ¡Cuidado con el overfitting aquí! Si vuestro modelo aprende muy bien de estos datos pero hay overfitting cuando le pasemos los datos desconocidos de `test.csv` nos arriesgamos a que digamos, no salga lo esperado.

### 1. Entrena dicho modelo con TODOS tus datos de train, esto es con `train.csv` al completo.


**CON LAS TRANSFORMACIONES QUE LE HAYAS REALIZADO A `X` INCLUÍDAS.**


Véase:
- Estandarización/Normalización
- Eliminación de Outliers
- Eliminación de columnas
- Creación de columnas nuevas
- Gestión de valores nulos
- Y un largo etcétera de técnicas que como Data Scientist hayas considerado las mejores para tu dataset.

In [48]:
# Ya he preprocesado el conjunto de test de la misma manera que el conjunto de entrenamiento previamente

# Preprocesar el conjunto de test de la misma manera que el conjunto de entrenamiento
X_pred = test.drop(columns=['laptop_ID', 'Product', 'Memory', 'Weight'])
X_pred = pd.get_dummies(X_pred, drop_first=True)
X_pred = X_pred.reindex(columns=X.columns, fill_value=0)

### 2. Carga los datos de `test.csv` para predecir.

In [49]:
# ya los cargué al principio

**¿Por qué puede dar error?** 

IMPORTANTE: APLICAR A ESTOS DATOS LO MISMO QUE HAYÁIS APLICADO A LOS DATOS DE ENTRENAMIENTO

- SI EL ARRAY CON EL QUE HICISTEIS `.fit()` ERA DE 4 COLUMNAS, PARA `.predict()` DEBEN SER LAS MISMAS
- SI AL ARRAY CON EL QUE HICISTEIS `.fit()` LO NORMALIZASTEIS, PARA `.predict()` DEBÉIS NORMALIZARLO
- TODO IGUAL SALVO BORRAR FILAS, EL NÚMERO DE ROWS SE DEBE MANTENER EN ESTE SET, PUES LA PREDICCIÓN DEBE TENER 391 FILAS, SI O SI

**Entonces, si al cargar los datos de train usé `index_col=0` para que utilizara la primera columna del conjunto de datos como índice, ¿tendré que hacerlo también para el conjunto `test.csv`?**

In [50]:
# ¿Qué opináis?
# Sí

![](https://www.tierraljelechu.com/web/img/wiki_up/1.996-SorpresaDto.-1-Red.jpg)

### 3. AHORA puedo hacer la predicción que será lo que subirás a Kaggle. 

**¿Qué es lo que subirás a Kaggle?**

In [52]:
'''# Realizar predicciones Regresion lineal
predictions_lr = model.predict(X_pred)
#predictions'''

'# Realizar predicciones Regresion lineal\npredictions_lr = model.predict(X_pred)\n#predictions'

In [ ]:
#predictions = model.predict(X_pred)
#predictions

In [ ]:
'''# Realizar predicciones LGBM
predictions_lgbm = model_lgbm.predict(X_pred)'''

In [ ]:
'''# Realizar predicciones Random Forest
predictions_rf = model_rf.predict(X_pred)'''

In [53]:
# Realizar predicciones con el mejor modelo
predictions = best_model.predict(X_pred)

**¡PERO! Para subir a Kaggle la predicción, ésta tendrá que tener una forma específica y no valdrá otra.**

En este caso, la **MISMA** forma que `sample_submission.csv`. 

In [57]:
sample = pd.read_csv(".\desafio-de-prediccion-de-precios-de-portatiles/sample_submission.csv")

In [66]:
sample.head()

,id,Price_euros
0,1014,752.0
1,845,499.0
2,1151,1747.0
3,1265,245.0
4,573,1179.0


In [67]:
sample.shape

(391, 2)

### 4. Mete tus predicciones en un dataframe. 

En este caso, la **MISMA** forma que `sample_submission.csv`. 

In [ ]:
'''# Crear el dataframe para la submission (Regresion Lineal)
submission_lr = sample.copy()
submission_lr['Price_euros'] = predictions_lr'''

In [ ]:
'''submission_lgbm = sample.copy()
submission_lgbm['Price_euros'] = predictions_lgbm'''

In [ ]:
'''submission_rf = sample.copy()
submission_rf['Price_euros'] = predictions_rf'''

In [73]:
# Crear el dataframe para la submission
submissiontest = pd.DataFrame({'id': test['id'], 'Price_euros': predictions})

# Guardar el archivo de submission
submissiontest.to_csv("submission_best_model.csv", index=False)

In [63]:
# Crear el dataframe para la submission
#submissiontest = test.copy()
#submissiontest['id', 'Price_euros'] = predictions

In [74]:
submissiontest

,id,Price_euros
0,181,2213.082520
1,708,855.840820
2,862,402.987640
3,1064,1326.610962
4,702,1144.244873
...,...,...
386,1281,1099.262085
387,524,1712.746216
388,1015,473.573029
389,1236,424.958649


### 5. Pásale el CHEQUEATOR para comprobar que efectivamente está listo para subir a Kaggle.

In [75]:
def chequeator(df_to_submit):
    """
    Esta función se asegura de que tu submission tenga la forma requerida por Kaggle.
    
    Si es así, se guardará el dataframe en un `csv` y estará listo para subir a Kaggle.
    
    Si no, LEE EL MENSAJE Y HAZLE CASO.
    
    Si aún no:
    - apaga tu ordenador, 
    - date una vuelta, 
    - enciendelo otra vez, 
    - abre este notebook y 
    - leelo todo de nuevo. 
    Todos nos merecemos una segunda oportunidad. También tú.
    """
    if df_to_submit.shape == sample.shape:
        if df_to_submit.columns.all() == sample.columns.all():
            if df_to_submit.id.all() == sample.id.all():
                print("You're ready to submit!")
                submissiontest.to_csv("submission.csv", index = False) #muy importante el index = False
                #urllib.request.urlretrieve("https://i.kym-cdn.com/photos/images/facebook/000/747/556/27a.jpg", "gfg.png")     
                #img = Image.open("gfg.png")
                #img.show()   
            else:
                print("Check the ids and try again")
        else:
            print("Check the names of the columns and try again")
    else:
        print("Check the number of rows and/or columns and try again")
        print("")


In [ ]:
#chequeator(submission_lr) # submission es el nombre que le habríamos puesto a nuestro .csv con los valores que me salieron en la predicción

In [76]:
chequeator(submissiontest)

You're ready to submit!


In [ ]:
'''chequeator(submission_rf)'''

'chequeator(submission_rf)'

In [ ]:
#chequeator(submission_lgbm)